In [1]:
import sys,os,re,joblib
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
sys.path.append("../")
import tools.funclib as funclib
import numpy as np
import pandas as pd

from keras.models import load_model
from tools.Attention import Attention
from tkinter import _flatten
import config as cfg
import benchmark_common as bcommon
import tools.embedding_esm as esmebd

from pandarallel import pandarallel 
pandarallel.initialize()

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. 读入数据

In [2]:
data_new = pd.read_csv('/home/shizhenkun/codebase/DMLF/data/case_science/new.csv', sep='\t').rename(columns={'Entry': 'id', 'EC number':'ec', 'Sequence':'seq'})
data_price = pd.read_csv('/home/shizhenkun/codebase/DMLF/data/case_science/price.csv', sep='\t').rename(columns={'Entry': 'id', 'EC number':'ec', 'Sequence':'seq'})
data_price.head(3)

,id,ec,seq
0,WP_063460136,5.3.1.7,MAIPPYPDFRSAAFLRQHLRATMAFYDPVATDASGGQFHFFLDDGT...
1,WP_063462980,4.2.1.43,VPTTFHEDGTLDLDSQKRCLDFMIDAGVDGVCILANFSEQFSLSDA...
2,WP_063462990,1.1.1.48,LIDCNIDMTQLFAPSSSSTDATGAPQGLAKFPSLQGRAVFVTGGGS...


## 3. 写入fasta

In [4]:
funclib.table2fasta(table=data_price, file_out='/home/shizhenkun/codebase/DMLF/data/case_science/price.fasta')
funclib.table2fasta(table=new, file_out='/home/shizhenkun/codebase/DMLF/data/case_science/new.fasta')

Write finished


## 4 运行预测

In [ ]:
! python production.py -i /home/shizhenkun/codebase/DMLF/data/case_science/new.fasta -o /home/shizhenkun/codebase/DMLF/results/case_science/new_res.tsv -mode r -topk 20
! python production.py -i /home/shizhenkun/codebase/DMLF/data/case_science/price.fasta -o /home/shizhenkun/codebase/DMLF/results/case_science/price_res.tsv -mode r -topk 20


## 5. 读入预测结果

In [6]:
new_res= pd.read_csv('/home/shizhenkun/codebase/DMLF/results/case_science/new_res.tsv', sep='\t')
price_res = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/case_science/price_res.tsv', sep='\t')

## 6. 分析结果

In [9]:
data_new.merge( new_res, how='left', on='id').to_excel('/home/shizhenkun/codebase/DMLF/results/case_science/new_res_r.xlsx', index=None)

In [10]:
data_price.merge(price_res, how='left' , on='id').to_excel('/home/shizhenkun/codebase/DMLF/results/case_science/price_res_r.xlsx', index=None)